# Create an initial dataset with elements from COCO dataset 
# and elements from densepose dataset condensing left-right in only one class
## all body parts included

## corresponds with version 14


# Getting data

In [1]:
import cv2
import json
from json import JSONEncoder
import pandas as pd
import numpy as np
import pycocotools.mask as mask_util

from tqdm import tqdm

In [2]:
version = 14

In [3]:
with open('/home/ubuntu/dormakaba/coco/annotations/instances_train2014.json') as json_file:
    data = json.load(json_file)

In [4]:
with open('/home/ubuntu/dormakaba/coco/annotations/densepose_coco_2014_train.json') as json_file:
    person_data = json.load(json_file)

In [5]:
data_df = pd.json_normalize(data)

In [6]:
person_data_df = pd.json_normalize(person_data)

In [7]:
data_df.keys()

Index(['images', 'licenses', 'annotations', 'categories', 'info.description',
       'info.url', 'info.version', 'info.year', 'info.contributor',
       'info.date_created'],
      dtype='object')

In [8]:
person_data_df.keys()

Index(['images', 'annotations', 'categories'], dtype='object')

In [9]:
# esta es la lista de imágenes anotadas con personas
# está ok
person_data_df['images'][0][0].keys()

dict_keys(['license', 'file_name', 'coco_url', 'height', 'width', 'date_captured', 'flickr_url', 'id'])

In [10]:
print(person_data_df['categories'][0])

[{'supercategory': 'person', 'id': 1, 'name': 'person', 'keypoints': ['nose', 'left_eye', 'right_eye', 'left_ear', 'right_ear', 'left_shoulder', 'right_shoulder', 'left_elbow', 'right_elbow', 'left_wrist', 'right_wrist', 'left_hip', 'right_hip', 'left_knee', 'right_knee', 'left_ankle', 'right_ankle'], 'skeleton': [[16, 14], [14, 12], [17, 15], [15, 13], [12, 13], [6, 12], [7, 13], [6, 7], [6, 8], [7, 9], [8, 10], [9, 11], [2, 3], [1, 2], [1, 3], [2, 4], [3, 5], [4, 6], [5, 7]]}]


# Generating classes

In [11]:
COCO_CLASSES = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
                'train', 'truck', 'boat', 'traffic light', 'fire hydrant',
                'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog',
                'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe',
                'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
                'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat',
                'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
                'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl',
                'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot',
                'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
                'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop',
                'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven',
                'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase',
                'scissors', 'teddy bear', 'hair drier', 'toothbrush']
# THE NEW CUSTOM CLASSES CONDENSE THE LEFT-RIGHT IN ONLY ONE CAT
CUSTOM_CLASSES = ['person', 'bicycle', 'cat', 'dog', 'backpack', 'umbrella', 'handbag', 'suitcase', 
                'sports ball', 'baseball bat', 'skateboard', 'tennis racket', 'bottle', 'wine glass', 
                'sandwich', 'chair', 'laptop', 'cell phone', 'book', 'clock', 'scissors', 'teddy bear',
                'torso', 'hand', 'foot', 'upleg', 'lowleg', 'uparm', 'lowarm', 'head']

#                'torso', 'hand1', 'hand2', 'foot1', 'foot2', 'upleg1', 'upleg2', 'lowleg1', 'lowleg2',
#                'uparm1', 'uparm2', 'lowarm1','lowarm2', 'head']

# ORIGINAL ELEMENTS IN DENSE POSE
# ['nose', 'left_eye', 'right_eye', 'left_ear', 'right_ear', 
# 'left_shoulder', 'right_shoulder', 'left_elbow', 'right_elbow', 
# 'left_wrist', 'right_wrist', 'left_hip', 'right_hip', 
# 'left_knee', 'right_knee', 'left_ankle', 'right_ankle']


FROM_COCO_CLASSES = ['person', 'bicycle', 'cat', 'dog', 'backpack', 'umbrella', 'handbag', 'suitcase', 
                'sports ball', 'baseball bat', 'skateboard', 'tennis racket', 'bottle', 'wine glass', 
                'sandwich', 'chair', 'laptop', 'cell phone', 'book', 'clock', 'scissors', 'teddy bear']

#DENSEPOSE_CLASSES = ['torso', 'hand1', 'hand2', 
#                    'foot1', 'foot2', 'upleg1', 'upleg2', 
#                    'lowleg1', 'lowleg2','uparm1', 'uparm2', 
#                    'lowarm1','lowarm2', 'head']

DENSEPOSE_CLASSES = ['torso', 'hand',  
                        'foot', 'upleg', 
                        'lowleg', 'uparm', 
                        'lowarm', 'head']
COCO_LABEL_MAP = {1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8,
                  9: 9, 10: 10, 11: 11, 13: 12, 14: 13, 15: 14, 16: 15, 17: 16,
                  18: 17, 19: 18, 20: 19, 21: 20, 22: 21, 23: 22, 24: 23, 25: 24,
                  27: 25, 28: 26, 31: 27, 32: 28, 33: 29, 34: 30, 35: 31, 36: 32,
                  37: 33, 38: 34, 39: 35, 40: 36, 41: 37, 42: 38, 43: 39, 44: 40,
                  46: 41, 47: 42, 48: 43, 49: 44, 50: 45, 51: 46, 52: 47, 53: 48,
                  54: 49, 55: 50, 56: 51, 57: 52, 58: 53, 59: 54, 60: 55, 61: 56,
                  62: 57, 63: 58, 64: 59, 65: 60, 67: 61, 70: 62, 72: 63, 73: 64,
                  74: 65, 75: 66, 76: 67, 77: 68, 78: 69, 79: 70, 80: 71, 81: 72,
                  82: 73, 84: 74, 85: 75, 86: 76, 87: 77, 88: 78, 89: 79, 90: 80}
# this map relates body part position to unique identifier (avoiding left-right problem)
PERSON_LABEL_MAP = {0:0, 1: 1, 2: 1, 
                    3: 2, 4: 2, 5: 3, 6: 3, 
                    7: 4, 8: 4, 9: 5, 10: 5, 
                    11: 6, 12: 6, 13: 7}

In [12]:
REVERSE_COCO_LABEL_MAP = {}
for key, value in COCO_LABEL_MAP.items():
    REVERSE_COCO_LABEL_MAP[value] = key

In [17]:
print(data_df['categories'][0][0])

custom_categories = []
for cat_data in data_df['categories'][0]:
    if cat_data['name'] in CUSTOM_CLASSES:
        custom_categories.append(cat_data)

for nc in DENSEPOSE_CLASSES:
    c = {'supercategory': 'person', 'id': 1, 'name': nc}
    custom_categories.append(c)

# this is the list of the new categories
print(custom_categories)
print()
print(len(CUSTOM_CLASSES), "-", len(custom_categories))
cc_id = 1
cc_list = [0]*(len(CUSTOM_CLASSES)+1)
for cc in custom_categories:
    cc['id'] = cc_id
    cc_list[cc_id] = cc['name']
    cc_id += 1

# the list of the names of the categories
print(cc_list)

{'supercategory': 'person', 'id': 1, 'name': 'person'}
[{'supercategory': 'person', 'id': 1, 'name': 'person'}, {'supercategory': 'vehicle', 'id': 2, 'name': 'bicycle'}, {'supercategory': 'animal', 'id': 3, 'name': 'cat'}, {'supercategory': 'animal', 'id': 4, 'name': 'dog'}, {'supercategory': 'accessory', 'id': 5, 'name': 'backpack'}, {'supercategory': 'accessory', 'id': 6, 'name': 'umbrella'}, {'supercategory': 'accessory', 'id': 7, 'name': 'handbag'}, {'supercategory': 'accessory', 'id': 8, 'name': 'suitcase'}, {'supercategory': 'sports', 'id': 9, 'name': 'sports ball'}, {'supercategory': 'sports', 'id': 10, 'name': 'baseball bat'}, {'supercategory': 'sports', 'id': 11, 'name': 'skateboard'}, {'supercategory': 'sports', 'id': 12, 'name': 'tennis racket'}, {'supercategory': 'kitchen', 'id': 13, 'name': 'bottle'}, {'supercategory': 'kitchen', 'id': 14, 'name': 'wine glass'}, {'supercategory': 'food', 'id': 15, 'name': 'sandwich'}, {'supercategory': 'furniture', 'id': 16, 'name': 'chair

In [18]:
coco_category_id_list = [0]*len(cc_list)
for cat in data_df['categories'][0]:
    if cat['name'] in cc_list:
        pos = cc_list.index(cat['name'])
        coco_category_id_list[pos] = cat['id']

# this is the list of the ids of the new categories when exists in COCO
# the ones that not exist in COCO comes from DENSEPOSE
print(coco_category_id_list)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 0, 0, 0, 0, 0, 0, 0, 0]


In [19]:
# this variable has the coco category id in the position of the custom category
custom_category_coco_category = [0]*len(cc_list)
for i, cat_name in enumerate(COCO_CLASSES):
    if cat_name in cc_list:
        cc_pos = cc_list.index(cat_name)
        coco_cat_id = REVERSE_COCO_LABEL_MAP[i+1]
        custom_category_coco_category[cc_pos] = coco_cat_id
        # print(i, cat_name, cc_pos, coco_cat_id)

print(custom_category_coco_category)

[0, 1, 2, 17, 18, 27, 28, 31, 33, 37, 39, 41, 43, 44, 46, 54, 62, 73, 77, 84, 85, 87, 88, 0, 0, 0, 0, 0, 0, 0, 0]


# looking at images
## Those are the images I'll use in the dataset that come from 
## the DENSEPOSE dataset

In [20]:
new_person_images = person_data_df['images'][0]
print( len(new_person_images), new_person_images[0].keys())

26437 dict_keys(['license', 'file_name', 'coco_url', 'height', 'width', 'date_captured', 'flickr_url', 'id'])


In [21]:
images_person_data_df = pd.DataFrame(person_data_df['images'][0])

In [22]:
a = images_person_data_df[images_person_data_df['id']==262145]
print(a['height'][0], a['width'][0])

427 640


In [23]:
# list of all image ids from DENSEPOSE
new_person_images_list = [a['id'] for a in person_data_df['images'][0]]
len(new_person_images_list)

26437

In [24]:
# this is the list of all the images id in coco dataset
all_images_list = [a['id'] for a in data_df['images'][0]]
len(all_images_list)

82783

# looking at annotations

In [25]:
# person annotations
# extraer segmentation: anotaciòn individual de cada persona detectada
# extrater dp_masks: e insertar una anotación por cada uno de los 14 elementos de la persona que tienen anotaciones
# si se puede, transformar el mapa de bits en 
print(person_data_df['annotations'][0][0].keys())
print(person_data_df['annotations'][0][0]['dp_masks'])

dict_keys(['segmentation', 'num_keypoints', 'dp_masks', 'area', 'dp_I', 'iscrowd', 'keypoints', 'dp_x', 'dp_U', 'image_id', 'dp_V', 'bbox', 'category_id', 'dp_y', 'id'])
[{'counts': '\\Qa03k7200M210N110N1N2100000O1000O0010O2O0O10001O0O1N200O2O001O0O1O?B0O:G0O;F017H5K011N001O010O2N001O010O002N001O1O011N001O010OO1000001O000O2O00000000001O00001O001N1000001O0O1000001N101O00001N101O002N1O002N002N002M102N1N103M001O002M2O003M001N102N2N002N001O002M3N002N002M101O002N2M102N001O002N2N002M102N001OUU9', 'size': [256, 256]}, {'counts': 'lk5110l70TH0j7400O2O0NNZHNf72ZHNe77O01003M0O010O10O100000O10000O01000000O10000O10O1001O001N101M3N101M20[\\_1', 'size': [256, 256]}, {'counts': 'i33m70L5O0O2O0O100O1OL`HIa77_HI_79_HIa77_HI`77`HK_75aHK_7:1N20OO200000000000O1000O10O0101N2N101O001N102M`Tf1', 'size': [256, 256]}, [], {'counts': 'e_Y12m73N0O2O002N0O10000000000000000000000000000000M2L50_P`0', 'size': [256, 256]}, {'counts': 'im[14l70N110L400M3N200M300M300M3N200M300L400N2M4O0M@c00ON30NN40ON0120ON30NO30NN3O20

In [26]:
print(person_data_df['annotations'][0][0])

{'segmentation': [[453, 292.1, 457, 253.1, 439, 245.1, 438, 215.1, 439, 198.1, 420, 223.1, 414, 233.1, 401, 227.1, 400, 226.1, 398, 229.1, 391, 231.1, 387, 213.1, 399, 203.1, 404, 200.1, 413, 194.1, 418, 186.1, 408, 181.1, 415, 154.1, 418, 142.1, 419, 127.1, 422, 125.1, 419, 120.1, 412, 122.1, 407, 112.1, 402, 105.1, 389, 113.1, 390, 105.1, 395, 100.1, 395, 97.1, 398, 83.1, 407, 72.1, 417, 71.1, 424, 72.1, 428, 73.1, 436, 80.1, 441, 90.1, 446, 96.1, 456, 101.1, 472, 110.1, 480, 113.1, 493, 123.1, 499, 136.1, 504, 147.1, 509, 167.1, 515, 182.1, 531, 205.1, 532, 218.1, 525, 229.1, 514, 246.1, 499, 283.1, 499, 307.1, 499, 323.1, 499, 343.1, 505, 367.1, 505, 380.1, 505, 381.1, 486, 387.1, 482, 392.1, 479, 393.1, 469, 363.1, 453, 343.1, 451, 339.1, 454, 321.1, 453, 312.1, 460, 313.1, 458, 298.1, 452, 293.1]], 'num_keypoints': 15, 'dp_masks': [{'counts': '\\Qa03k7200M210N110N1N2100000O1000O0010O2O0O10001O0O1N200O2O001O0O1O?B0O:G0O;F017H5K011N001O010O2N001O010O002N001O1O011N001O010OO1000001O0

# generating annotations

In [27]:
def GetDensePoseMasks(Polys):
    mask_list = [0]*15
    for i in range(1,15):
        MaskGen = np.zeros([256,256])
        if(Polys[i-1]):
            current_mask = mask_util.decode(Polys[i-1])
            MaskGen[current_mask>0] = 1
        mask_list[i-1]=MaskGen
    return mask_list

In [28]:
def GetDensePoseMask(encoded_mask):
    MaskGen = np.zeros([256,256])
    current_mask = mask_util.decode(encoded_mask)
    MaskGen[current_mask>0] = 1
    return MaskGen

In [29]:
def get_poligons(mask):
    c = cv2.findContours(mask.astype(np.uint8), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = [0]*2*len(c[0][0].tolist())
    for i,x in enumerate(c[0][0].tolist()):
        contours[2*i]=int(x[0][0])
        contours[2*i+1]= int(x[0][1])
    return contours

In [30]:
def CreateMaskBBox(mask, bbr, image_shape):
    final_mask = np.zeros([image_shape[1],image_shape[0]])
    x1 = int(bbr[0])
    y1 = int(bbr[1])
    x2 = int(bbr[0]+bbr[2])
    y2 = int(bbr[1]+bbr[3])
    x2 = min( [ x2, image_shape[0] ] )
    y2 = min( [ y2, image_shape[1] ] )
    # print(bbr, (x1,y1,x2,y2), (int(x2-x1),int(y2-y1)), mask.shape)
    resized_mask = cv2.resize( mask, (int(x2-x1),int(y2-y1)) ,interpolation=cv2.INTER_NEAREST)
    final_mask[y1:y2,x1:x2] = resized_mask
    az = np.nonzero(final_mask)
    bby1,bby2,bbx1,bbx2=np.min(az[0]), np.max(az[0]), np.min(az[1]), np.max(az[1])
    contours = get_poligons(final_mask.astype(np.uint8))
    encoded_final_mask = mask_util.encode(np.asfortranarray(final_mask.astype(np.uint8)))
    mask_area = int(mask_util.area(encoded_final_mask))
    # print((bbx1, bby1, bbx2-bbx1, bby2-bby1),encoded_final_mask)
    encoded_final_mask['counts'] = encoded_final_mask['counts'].decode("utf-8")
    return final_mask, (bbx1, bby1, bbx2-bbx1, bby2-bby1), mask_area, encoded_final_mask, contours

In [31]:
new_person_ann = []
for annotation in tqdm(person_data_df['annotations'][0]):
    # print(annotation['image_id'], annotation['bbox'])
    if annotation['iscrowd']==0 and annotation['image_id'] in new_person_images_list:
        ann_segment = annotation['segmentation']
        ann_bbox = [int(annotation['bbox'][0]),int(annotation['bbox'][1]),int(annotation['bbox'][2]),int(annotation['bbox'][3])]
        ann_area = annotation['area']
        ann_image_id = annotation['image_id']
        ann_id = annotation['id']
        ann_cat_id = 1 # only person cat so must be 1
        new_data = {'segmentation': ann_segment,
                    'area': ann_area,
                    'image_id': ann_image_id,
                    'id': ann_id,
                    'category_id': ann_cat_id,
                    'bbox': ann_bbox,
                    'iscrowd': 0}
        new_person_ann.append(new_data)
        a = images_person_data_df[images_person_data_df['id']==ann_image_id]
        image_size = [a['width'].values[0],a['height'].values[0]]

        if 'dp_masks' in annotation:
            for i, ann_mask in enumerate(annotation['dp_masks']):
                if len(ann_mask) > 1:
                    ann_mask_decoded = GetDensePoseMask(ann_mask)
                    mask, bbox, ann_area, encoded_mask, contours = CreateMaskBBox(ann_mask_decoded, annotation['bbox'], image_size )
                    ann_bbox = [int(bbox[0]),int(bbox[1]),int(bbox[2]),int(bbox[3])]
                    ann_segment = encoded_mask
                    ann_image_id = annotation['image_id']
                    ann_id = -1
                    # for reusing ids I'll prepare a first approach 
                    # with a cat id dictionary
                    ann_cat_id = 23+PERSON_LABEL_MAP[i]
                    if len(contours)>4:
                        new_data = {'segmentation': [contours], # encoded_mask,
                                    'area': ann_area,
                                    'image_id': ann_image_id,
                                    'id': ann_id,
                                    'category_id': ann_cat_id,
                                    'bbox': ann_bbox,
                                    'iscrowd': 0}
                        new_person_ann.append(new_data)

100%|██████████| 100403/100403 [33:20<00:00, 50.19it/s]


In [32]:
# check that the list of annotated image_ids has the len len as densepose image id's
new_person_images_annotated_list = [a['image_id'] for a in new_person_ann]
print( len(new_person_images_annotated_list), len(set(new_person_images_annotated_list)), len(set(new_person_images_list)))

474298 26437 26437


# till here I have all human data and parts segmented and annotated
# next step is to include the other classes

In [33]:
new_data_ann = []
new_data_image_list = []
new_data_images = []
for annotation in tqdm(data_df['annotations'][0]):
    if annotation['category_id'] in custom_category_coco_category and annotation['category_id']>1:
        ann_segment = annotation['segmentation']
        ann_bbox = [int(annotation['bbox'][0]),int(annotation['bbox'][1]),int(annotation['bbox'][2]),int(annotation['bbox'][3])]
        ann_area = annotation['area']
        ann_image_id = annotation['image_id']
        ann_id = annotation['id']
        ann_iscrowd = annotation['iscrowd']
        ann_cat_id = custom_category_coco_category.index(annotation['category_id'])
        new_data = {'segmentation': ann_segment,
                    'area': ann_area,
                    'image_id': ann_image_id,
                    'id': ann_id,
                    'category_id': ann_cat_id,
                    'bbox': ann_bbox,
                    'iscrowd': ann_iscrowd}
        # new_person_ann.append(new_data)
        new_data_image_list.append(ann_image_id)

100%|██████████| 604907/604907 [00:01<00:00, 493578.61it/s]


In [34]:
new_data_images = [annotation for annotation in data_df['images'][0] if annotation['id'] in new_data_image_list]

# Saving human parts data

In [35]:

new_human_part_data = {
    'images': new_person_images, 
    'annotations': new_person_ann, 
    'categories': custom_categories
}

In [36]:
class MyEncoder(JSONEncoder):
    def default(self, o):
        return o.__dict__
new_human_part_data_json = MyEncoder().encode(new_human_part_data)

In [37]:
with open(f'/home/ubuntu/dormakaba/coco/annotations/custom_ann_human_parts_train2014_v{version}.json', 'w') as f:
    json.dump(new_human_part_data_json, f)

# Saving non human data

In [38]:

new_non_human_data = {
    'images': new_data_images, 
    'annotations': new_data_ann, 
    'categories': custom_categories
}

In [39]:
print(len(new_data_images))

59196


In [40]:
class MyEncoder(JSONEncoder):
    def default(self, o):
        return o.__dict__
new_non_human_data_json = MyEncoder().encode(new_non_human_data)

In [41]:
with open(f'/home/ubuntu/dormakaba/coco/annotations/custom_ann_non_human_train2014_v{version}.json', 'w') as f:
    json.dump(new_non_human_data_json, f)

# updating indices

In [42]:
final_images = new_person_images + new_data_images
final_annotations = new_person_ann + new_data_ann

In [43]:
nil = []
for image in tqdm(final_images):
    if image["id"] not in all_images_list:
        nil.append[image["id"]]
len(nil)

100%|██████████| 85633/85633 [00:51<00:00, 1666.59it/s]


0

In [44]:
final_annotations[0].keys()

dict_keys(['segmentation', 'area', 'image_id', 'id', 'category_id', 'bbox', 'iscrowd'])

In [45]:
for i in tqdm(range(len(final_annotations))):
    final_annotations[i]['id'] = i+1

100%|██████████| 474298/474298 [00:00<00:00, 1135902.95it/s]


In [46]:
nil = []
for image in tqdm(final_images):
    if image["id"] not in all_images_list:
        nil.append[image["id"]]
len(nil)

100%|██████████| 85633/85633 [00:51<00:00, 1669.84it/s]


0

# saving all data

In [47]:
# IN THE version_02
new_person_data = {
    'images': final_images, 
    'annotations': final_annotations, 
    'categories': custom_categories
}

In [48]:
class MyEncoder(JSONEncoder):
    def default(self, o):
        return o.__dict__
new_person_data_json = MyEncoder().encode(new_person_data)

In [49]:
with open(f'/home/ubuntu/dormakaba/coco/annotations/custom_ann_train2014_v{version}.json', 'w') as f:
    json.dump(new_person_data_json, f)